<a href="https://colab.research.google.com/github/vigilant-umbrella/automatic-quality-estimation/blob/main/knn_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [ ]:
os.chdir('./drive/My Drive/wikiHow')

# Loading Data

In [ ]:
df = pd.read_csv('wikihow.csv')
df.head()

,character_count,word_count,method_count,mean_method_size,mean_paragraph_size,size_largest_method,size_shortest_method,std_method_size,step_count,mean_steps_per_method,...,Kincaid,ARI,Coleman_Liau,FleschReadingEase,GunningFogIndex,LIX,SMOGIndex,RIX,DaleChallIndex,percent_helpful
0,7107,1508,2,3357.500000,379.823529,4321,2394,963.500000,16,8.000000,...,7.523760,8.159750,9.908877,67.453457,10.820887,36.866542,10.376433,3.264706,10.086871,78
1,5180,1176,2,2419.500000,317.000000,3171,1668,751.500000,14,7.000000,...,4.951055,5.505902,7.708411,81.614246,8.897157,33.807519,9.052403,2.652632,9.111023,99
2,3822,902,3,1114.333333,263.333333,1521,881,288.598144,11,3.666667,...,6.020412,6.727450,7.309495,81.109399,9.797251,31.810200,9.310165,2.527273,8.038576,88
3,11614,2429,3,3604.333333,393.111111,4148,2832,561.058721,26,8.666667,...,7.798307,8.728719,10.376311,66.362137,11.577962,38.743177,10.914639,3.584906,9.965255,82
4,9633,2004,4,2310.500000,535.411765,2753,1922,329.196370,16,4.000000,...,8.716429,9.423549,10.661749,61.827048,13.316999,42.773535,12.116729,4.327869,9.123631,100


In [ ]:
df.shape

(19917, 45)

In [ ]:
X = df.drop('percent_helpful', axis=1)
y  = df['percent_helpful']

In [ ]:
X_full_train, X_test, y_full_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def create_pipeline():
    imputer = SimpleImputer(strategy='median')

    scaler = MinMaxScaler()

    model = KNeighborsRegressor(n_jobs=-1)

    model = TransformedTargetRegressor(
        regressor=model,
        transformer=MinMaxScaler()
    )

    pipeline = Pipeline(
        steps=[
            ('imputer', imputer),
            ('scaling', scaler),
            ('model', model)
        ]
    )

    return pipeline

In [ ]:
search_space = {
    'model__regressor__n_neighbors': [3, 5, 7],
    'model__regressor__weights': ['uniform', 'distance'],
    'model__regressor__p': [1 , 2, 3, 4, 5]
}

In [ ]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

searcher = GridSearchCV(
    create_pipeline(),
    search_space,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    cv=cv,
    verbose=5
)

searcher.fit(X_full_train, y_full_train)

Fitting 10 folds for each of 30 candidates, totalling 300 fits
[CV 3/10] END model__regressor__n_neighbors=3, model__regressor__p=1, model__regressor__weights=uniform;, score=-6.324 total time=   9.0s
[CV 3/10] END model__regressor__n_neighbors=3, model__regressor__p=1, model__regressor__weights=distance;, score=-6.331 total time=  10.7s
[CV 2/10] END model__regressor__n_neighbors=3, model__regressor__p=2, model__regressor__weights=uniform;, score=-6.176 total time=   6.7s


/home/aniketsharma00411/anaconda3/envs/python3.8/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 2/10] END model__regressor__n_neighbors=3, model__regressor__p=1, model__regressor__weights=uniform;, score=-6.185 total time=   9.7s
[CV 5/10] END model__regressor__n_neighbors=3, model__regressor__p=1, model__regressor__weights=distance;, score=-6.292 total time=  11.0s
[CV 4/10] END model__regressor__n_neighbors=3, model__regressor__p=2, model__regressor__weights=uniform;, score=-6.368 total time=   7.3s
[CV 6/10] END model__regressor__n_neighbors=3, model__regressor__p=1, model__regressor__weights=uniform;, score=-6.374 total time=   9.0s
[CV 9/10] END model__regressor__n_neighbors=3, model__regressor__p=1, model__regressor__weights=uniform;, score=-6.433 total time=  10.1s
[CV 7/10] END model__regressor__n_neighbors=3, model__regressor__p=1, model__regressor__weights=distance;, score=-6.294 total time=   8.8s
[CV 7/10] END model__regressor__n_neighbors=3, model__regressor__p=2, model__regressor__weights=uniform;, score=-6.394 total time=   5.8s
[CV 7/10] END model__regressor__

GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('imputer',
                                        SimpleImputer(strategy='median')),
                                       ('scaling', MinMaxScaler()),
                                       ('model',
                                        TransformedTargetRegressor(regressor=KNeighborsRegressor(n_jobs=-1),
                                                                   transformer=MinMaxScaler()))]),
             n_jobs=-1,
             param_grid={'model__regressor__n_neighbors': [3, 5, 7],
                         'model__regressor__p': [1, 2, 3, 4, 5],
                         'model__regressor__weights': ['uniform', 'distance']},
             scoring='neg_mean_absolute_error', verbose=5)

In [ ]:
searcher.best_score_

-5.8752930880272825

In [ ]:
searcher.best_params_

{'model__regressor__n_neighbors': 7,
 'model__regressor__p': 5,
 'model__regressor__weights': 'uniform'}